In [1]:
# importa os pacotes necessários
import numpy as np
import os, gc, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score

from sklearn import svm
from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('kaggle/data_pca.csv')
df_labels = pd.read_csv('kaggle/data_labels.csv')

In [3]:
#df=(df-df.min())/(df.max()-df.min())

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df, df_labels.label.values, test_size = 0.3, random_state = 42)
del df
X_train.shape,X_test.shape

((35000, 2500), (15000, 2500))

# Logistic Regression
Utilizando regressão logistica para ter uma base de comparação com os demais modelos.

#### nomalizado antes e padding

In [5]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=2)
logreg.fit(X_train, y_train)

Wall time: 11min 35s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=2, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
# avalia predicao
y_pred = logreg.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck,accuracy,macro avg,weighted avg
f1-score,0.602941,0.633854,0.449190,0.375000,0.452778,0.439268,0.611780,0.610181,0.709022,0.627041,0.5532,0.551106,0.550898
precision,0.593812,0.622876,0.464718,0.393723,0.451836,0.452055,0.584577,0.609365,0.696629,0.624187,0.5532,0.549378,0.549320
recall,0.612356,0.645227,0.434667,0.357977,0.453723,0.427184,0.641638,0.610999,0.721863,0.629921,0.5532,0.553556,0.553200
support,1473.000000,1477.000000,1500.000000,1542.000000,1437.000000,1545.000000,1465.000000,1491.000000,1546.000000,1524.000000,0.5532,15000.000000,15000.000000


In [ ]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=4)
logreg.fit(X_train.iloc[:,:1500], y_train)

In [ ]:
# avalia predicao
y_pred = logreg.predict(X_test.iloc[:,:1500])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

# Linear SVM
Classificador SVM otimizado para o kernel linear, pode não dar o mesmo 
resultado se comparado ao sklearn.svm.SVC(), mas tem um tempo de execução menor

In [4]:
%%time
svclassifier = LinearSVC()
svclassifier.fit(X_train, y_train)

Wall time: 30min 47s


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [5]:
# avalia predicao
y_pred = svclassifier.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck,accuracy,macro avg,weighted avg
f1-score,0.574206,0.590769,0.399304,0.321023,0.437086,0.430493,0.573123,0.571429,0.656047,0.596301,0.517533,0.514978,0.514679
precision,0.549287,0.596685,0.417455,0.354788,0.437849,0.426126,0.553787,0.557398,0.659477,0.580025,0.517533,0.513288,0.513141
recall,0.601494,0.584970,0.382667,0.293126,0.436326,0.434951,0.593857,0.586184,0.652652,0.613517,0.517533,0.517974,0.517533
support,1473.000000,1477.000000,1500.000000,1542.000000,1437.000000,1545.000000,1465.000000,1491.000000,1546.000000,1524.000000,0.517533,15000.000000,15000.000000


# Polinomial SVM
A função polinomial de 3° grau é a escolhida para o kernel

In [6]:
%%time
polysvclassifier = SVC(kernel='poly', tol=0.002)
polysvclassifier.fit(X_train, y_train)

Wall time: 2h 6min 3s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.002, verbose=False)

In [7]:
# avalia predicao
y_pred = polysvclassifier.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck,accuracy,macro avg,weighted avg
f1-score,0.255061,0.139888,0.001330,0.0,0.189733,0.007727,0.020013,0.007853,0.201183,0.175869,0.1442,0.099866,0.099265
precision,0.500994,0.278226,0.250000,0.0,0.105215,0.750000,0.441176,0.162162,0.597444,0.398148,0.1442,0.348337,0.350159
recall,0.171079,0.093433,0.000667,0.0,0.964509,0.003883,0.010239,0.004024,0.120957,0.112861,0.1442,0.148165,0.144200
support,1473.000000,1477.000000,1500.000000,1542.0,1437.000000,1545.000000,1465.000000,1491.000000,1546.000000,1524.000000,0.1442,15000.000000,15000.000000
